<a href="https://colab.research.google.com/github/martinoscarrodriguez/Analisis_y_Visualizacion_TP/blob/main/TP_Parte1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>